# JobShop algorithm for single stage scheduling 

## Importing packages

In [53]:

import collections
import time
from ortools.sat.python import cp_model
import pandas as pd
import contextlib
import io
import sys
import warnings
warnings.filterwarnings("ignore")
# actual = pd.read_csv("jobshop_actual.csv", sep="\t")
me = pd.read_csv("jobshop_predicted_S2.csv", sep="\t")

## JobShop algorithm - JSS2



In [54]:
import collections
import time
from ortools.sat.python import cp_model

class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__solution_count = 0
        self.__start_time = time.time()

    def on_solution_callback(self):
        """Called at each new solution."""
        current_time = time.time()
        elapsed_time = current_time - self.__start_time
        print(
            "Solution %i, time = %f s, objective = %i"
            % (self.__solution_count, elapsed_time, self.ObjectiveValue())
        )
        self.__solution_count += 1

def flexible_jobshop_2(me):
    """solve a small flexible jobshop problem."""
    #data
    import pandas as pd
    df = me
    # run=pd.read_csv("jobshop_predicted_S2.csv", sep="\t", usecols=["Sample"])
    run = df[["Sample"]]
    run = run["Sample"].tolist()
    df = df.drop("Sample", axis=1)
    data = df.values.tolist()
    jobs = [[[(data[i][j], j//2) for j in range(0, len(data[i]), 2)], [(data[i][j], j//2) for j in range(1, len(data[i]), 2)]] for i in range(len(data))]
    # print(jobs)
    num_jobs = len(jobs)
    all_jobs = range(num_jobs)
    num_machines = 5
    all_machines = range(num_machines)

    # Model the flexible jobshop problem.
    model = cp_model.CpModel()

    horizon = 0
    for job in jobs:
        for task in job:
            max_task_duration = 0
            for alternative in task:
                max_task_duration = max(max_task_duration, alternative[0])
            horizon += max_task_duration

    print("Horizon = %i" % horizon)

    # Global storage of variables.
    intervals_per_resources = collections.defaultdict(list)
    starts = {}  # indexed by (job_id, task_id).
    presences = {}  # indexed by (job_id, task_id, alt_id).
    job_ends = []

    # Scan the jobs and create the relevant variables and intervals.
    for job_id in all_jobs:
        job = jobs[job_id]
        num_tasks = len(job)
        previous_end = None
        for task_id in range(num_tasks):
            task = job[task_id]

            min_duration = task[0][0]
            max_duration = task[0][0]

            num_alternatives = len(task)
            all_alternatives = range(num_alternatives)

            for alt_id in range(1, num_alternatives):
                alt_duration = task[alt_id][0]
                min_duration = min(min_duration, alt_duration)
                max_duration = max(max_duration, alt_duration)

            # Create main interval for the task.
            suffix_name = "_j%i_t%i" % (job_id, task_id)
            start = model.NewIntVar(0, horizon, "start" + suffix_name)
            duration = model.NewIntVar(
                min_duration, max_duration, "duration" + suffix_name
            )
            end = model.NewIntVar(0, horizon, "end" + suffix_name)
            interval = model.NewIntervalVar(
                start, duration, end, "interval" + suffix_name
            )

            # Store the start for the solution.
            starts[(job_id, task_id)] = start

            # Add precedence with previous task in the same job.
            if previous_end is not None:
                model.Add(start >= previous_end)
            previous_end = end

            # Create alternative intervals.
            if num_alternatives > 1:
                l_presences = []
                for alt_id in all_alternatives:
                    alt_suffix = "_j%i_t%i_a%i" % (job_id, task_id, alt_id)
                    l_presence = model.NewBoolVar("presence" + alt_suffix)
                    l_start = model.NewIntVar(0, horizon, "start" + alt_suffix)
                    l_duration = task[alt_id][0]
                    l_end = model.NewIntVar(0, horizon, "end" + alt_suffix)
                    l_interval = model.NewOptionalIntervalVar(
                        l_start, l_duration, l_end, l_presence, "interval" + alt_suffix
                    )
                    l_presences.append(l_presence)

                    # Link the primary/global variables with the local ones.
                    model.Add(start == l_start).OnlyEnforceIf(l_presence)
                    model.Add(duration == l_duration).OnlyEnforceIf(l_presence)
                    model.Add(end == l_end).OnlyEnforceIf(l_presence)

                    # Add the local interval to the right machine.
                    intervals_per_resources[task[alt_id][1]].append(l_interval)

                    # Store the presences for the solution.
                    presences[(job_id, task_id, alt_id)] = l_presence

                # Select exactly one presence variable.
                model.AddExactlyOne(l_presences)
            else:
                intervals_per_resources[task[0][1]].append(interval)
                presences[(job_id, task_id, 0)] = model.NewConstant(1)

        job_ends.append(previous_end)

    # Create machines constraints.
    for machine_id in all_machines:
        intervals = intervals_per_resources[machine_id]
        if len(intervals) > 1:
            model.AddNoOverlap(intervals)

    # Makespan objective
    makespan = model.NewIntVar(0, horizon, "makespan")
    model.AddMaxEquality(makespan, job_ends)
    model.Minimize(makespan)

    # Solve model
    solver = cp_model.CpSolver()
    solution_printer = SolutionPrinter()
    status = solver.solve(model, solution_printer)
    n = 0
    # Print final solution.
    print(status)
    if status in (cp_model.FEASIBLE, cp_model.OPTIMAL):
        print(f"Optimal objective value: {solver.objective_value}")
        for job_id in all_jobs:
            for task_id in range(len(jobs[job_id])):
                start_value = solver.Value(starts[(job_id, task_id)])
                machine = -1
                duration = -1
                selected = -1
                for alt_id in range(len(jobs[job_id][task_id])):
                    if solver.Value(presences[(job_id, task_id, alt_id)]):
                        duration = jobs[job_id][task_id][alt_id][0]
                        machine = jobs[job_id][task_id][alt_id][1]
                        selected = alt_id
                runner = run[n]
                print("%s_%i starts at %i (alt %i, machine %i, duration %i)" % (runner,task_id, start_value, selected, machine, duration))
            n = n + 1
        print("solve status: %s" % solver.StatusName(status))
        print("Optimal objective value: %i" % solver.ObjectiveValue())
        print("Statistics")
        print("  - conflicts : %i" % solver.NumConflicts())
        print("  - branches  : %i" % solver.NumBranches())
        print("  - wall time : %f s" % solver.WallTime())

# flexible_jobshop()
output_capture = io.StringIO()
# Redirect stdout to the StringIO object
with contextlib.redirect_stdout(output_capture):
    flexible_jobshop_2(me)

# Retrieve the captured output
output = output_capture.getvalue()
print(output)  


Horizon = 19252
Solution 0, time = 0.048542 s, objective = 10245
Solution 1, time = 0.053806 s, objective = 9918
Solution 2, time = 0.056134 s, objective = 9776
Solution 3, time = 0.058533 s, objective = 5835
Solution 4, time = 0.060454 s, objective = 5514
Solution 5, time = 0.061749 s, objective = 5062
Solution 6, time = 0.062974 s, objective = 4974
Solution 7, time = 0.064550 s, objective = 4973
Solution 8, time = 0.065261 s, objective = 4836
Solution 9, time = 0.066107 s, objective = 4790
Solution 10, time = 0.066827 s, objective = 4701
Solution 11, time = 0.068191 s, objective = 4591
Solution 12, time = 0.068924 s, objective = 4400
Solution 13, time = 0.071011 s, objective = 4399
Solution 14, time = 0.071711 s, objective = 4398
Solution 15, time = 0.072445 s, objective = 4379
Solution 16, time = 0.073181 s, objective = 4214
Solution 17, time = 0.073923 s, objective = 4203
Solution 18, time = 0.074768 s, objective = 4126
Solution 19, time = 0.075535 s, objective = 4035
Solution 20, 

In [55]:
start = output.find("ERR")
end = output.find("solve status")
output = output[start:end]
# import pdb; pdb.set_trace()
output = output.split("\n")
output = [x for x in output if x]
output = [x.split() for x in output]
output = [[x[0], x[-3].split(",")[0], x[-7].split(")")[0] ,x[-1].split(")")[0]] for x in output]
# convert the output to a dataframe
output = pd.DataFrame(output, columns=["Job", "Machine","Start", "Duration"])
# print(output)
# output_sorted = output.sort_values(by=["Machine","Start"])
output["Start"] = output["Start"].astype(int)
output["Duration"] = output["Duration"].astype(int)
output_sorted = output.groupby("Machine").apply(lambda x: x.sort_values("Start")).reset_index(drop=True)
print(output_sorted)
grouped=output_sorted.groupby("Machine").agg({'Duration':'sum'}).reset_index()
print(grouped)
t=output_sorted
#increment by 1 in all values of Machine column
t["Machine"] = t["Machine"].astype(int);t["Machine"] = t["Machine"] + 1; t = t[["Job","Machine"]];t
# group Job by Machine and make a list of Jobs of dictionary keeping machine as key
t_jobs = t.groupby('Machine')['Job'].apply(list).reset_index()
t_jobs = t_jobs.set_index('Machine').to_dict(); t_jobs = t_jobs["Job"]
print(t_jobs)
# remove job name from the dictionary
job_allocation=t_jobs;del t_jobs
# job_allocation
# Input ei - Execution plan for machine mi
# Output: Gives execution schedule for VMs and Executes the schedule on machines
    

            Job Machine  Start  Duration
0   ERR016294_0       0      0       313
1   ERR062953_0       0    313       449
2   ERR016326_0       0    762       574
3   ERR062973_0       0   1336       404
4   ERR062953_1       0   1740       440
5   ERR016326_1       0   2180       638
6   ERR022370_1       0   2818       272
7   ERR023220_0       1      0      1207
8   ERR022370_0       1   1207       252
9   ERR020276_1       1   1750      1702
10  ERR018550_0       2      0       453
11  ERR016338_0       2    453       542
12  ERR018550_1       2    995       438
13  ERR016338_1       2   1433       566
14  ERR016294_1       2   1999       319
15  ERR062973_1       2   2318       400
16  ERR020253_0       3      0      1767
17  ERR023220_1       3   1767      1213
18  ERR020276_0       4      0      1750
19  ERR020253_1       4   1767      1572
  Machine  Duration
0       0      3090
1       1      3161
2       2      2718
3       3      2980
4       4      3322
{1: ['ERR016294_0',

In [56]:
machines = [1,2,3,4,5]
for i in range(0,len(machines)):
    ei = job_allocation[machines[i]]
    with open("script_m{}.txt".format(machines[i]), "w") as f:
        f.write("BEGIN\n")
        for j in ei:
            # print(j) # This works only for two stages needs to be modified for more stages
            if j.endswith("1"):
                # print("second stage")
                f.write("WAIT " + j + "\n") # Here it wait and check if j_0 is completed
                f.write("EXEC " + j + "\n")
            else:
                # print("first stage")
                f.write("EXEC " + j + "\n")
                f.write("SIGNAL " + j.split("_")[0] + "_1\n") # this means j_0 is completed
        f.write("END\n")
for i in range(len(machines)):
    print(f"Jobs on Machine {machines[i]}: {job_allocation[machines[i]]}")

Jobs on Machine 1: ['ERR016294_0', 'ERR062953_0', 'ERR016326_0', 'ERR062973_0', 'ERR062953_1', 'ERR016326_1', 'ERR022370_1']
Jobs on Machine 2: ['ERR023220_0', 'ERR022370_0', 'ERR020276_1']
Jobs on Machine 3: ['ERR018550_0', 'ERR016338_0', 'ERR018550_1', 'ERR016338_1', 'ERR016294_1', 'ERR062973_1']
Jobs on Machine 4: ['ERR020253_0', 'ERR023220_1']
Jobs on Machine 5: ['ERR020276_0', 'ERR020253_1']


In [58]:
#ERR016294_1
import subprocess
import sys
import os
# def Wait_function():
# Call a bash script here
    
def signal_function(file_name):
    # bash script to create a file here, /mnt/nvme/ERR016294_1
    cmd = f"touch /mnt/nvme/{file_name}"
    run_set = subprocess.call(cmd, shell=True)
    print(f"Signal sent: {file_name}")
    return run_set

def exec_function(file_name):
    # bash script to create a file here, /mnt/nvme/ERR016294_1
    # call parabricks.sh -s command in this function
